# Causal language modeling

In [1]:
!pip install transformers datasets evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


## Load ELI5 dataset

In [2]:
from datasets import load_dataset
eli5 = load_dataset("eli5_category", split="train[:5000]")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/12.6k [00:00<?, ?B/s]

eli5_category.py:   0%|          | 0.00/4.17k [00:00<?, ?B/s]

The repository for eli5_category contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/eli5_category.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/91772 [00:00<?, ? examples/s]

Generating validation1 split:   0%|          | 0/5446 [00:00<?, ? examples/s]

Generating validation2 split:   0%|          | 0/2375 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5411 [00:00<?, ? examples/s]

In [3]:
eli5 = eli5.train_test_split(test_size=0.2)

In [4]:
# Inspect the dataset
eli5['train'][0]

{'q_id': '5oflgs',
 'title': 'Why commercials have a long and short version.',
 'selftext': "Many times I will see an ad for the first time and it is 30 seconds or so, then in the coming weeks that commercial will be cut to a shorter version, maybe 10 or 15 seconds. Sometimes it even has shots that weren't in the original. Anyone in advertising who could shed some light?",
 'category': 'Technology',
 'subreddit': 'explainlikeimfive',
 'answers': {'a_id': ['dcizcrv'],
  'text': ['Yep, it is a specific marketing strategy aimed at regular tv watchers. They start running a long commercial (1 minute) that hopefully does a really good job of being remember by the audience. This if often not hard because 1 minute commercials often stand out because they are so long already. They will run these for a few weeks, and then cut it down to a 30 second spot. This has the affect of you already recognizing the commercial, thus it doing its job, but costing less money as commercials are charged by run 

In [5]:
eli5['train'][0]['answers']['text']

['Yep, it is a specific marketing strategy aimed at regular tv watchers. They start running a long commercial (1 minute) that hopefully does a really good job of being remember by the audience. This if often not hard because 1 minute commercials often stand out because they are so long already. They will run these for a few weeks, and then cut it down to a 30 second spot. This has the affect of you already recognizing the commercial, thus it doing its job, but costing less money as commercials are charged by run time and audience size. Sometimes they will even reduce it to a 15 second spot after that. It is just a way to maximum people remembering the commercial while minimizing the cost.']

## Preprocess

In [7]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilgpt2")

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [8]:
eli5 = eli5.flatten()

In [9]:
eli5['train'][0]

{'q_id': '5oflgs',
 'title': 'Why commercials have a long and short version.',
 'selftext': "Many times I will see an ad for the first time and it is 30 seconds or so, then in the coming weeks that commercial will be cut to a shorter version, maybe 10 or 15 seconds. Sometimes it even has shots that weren't in the original. Anyone in advertising who could shed some light?",
 'category': 'Technology',
 'subreddit': 'explainlikeimfive',
 'answers.a_id': ['dcizcrv'],
 'answers.text': ['Yep, it is a specific marketing strategy aimed at regular tv watchers. They start running a long commercial (1 minute) that hopefully does a really good job of being remember by the audience. This if often not hard because 1 minute commercials often stand out because they are so long already. They will run these for a few weeks, and then cut it down to a 30 second spot. This has the affect of you already recognizing the commercial, thus it doing its job, but costing less money as commercials are charged by r

In [13]:
def preprocess_function(examples):
  return tokenizer([" ".join(x) for x in examples["answers.text"]])

In [14]:
tokenized_eli5 = eli5.map(
  preprocess_function,
  batched=True,
  num_proc=4,
  remove_columns=eli5["train"].column_names,
)

Map (num_proc=4):   0%|          | 0/4000 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (2653 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1580 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1913 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1808 > 1024). Running this sequence through the model will result in indexing errors


Map (num_proc=4):   0%|          | 0/1000 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1850 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1318 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1112 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1443 > 1024). Running this sequence through the model will result in indexing errors


In [20]:
tokenized_eli5

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 4000
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 1000
    })
})

In [27]:
print(len(tokenized_eli5['train']['input_ids']))
print([len(tokenized_eli5['train']['input_ids'][i]) for i in range(5)])

4000
[140, 133, 219, 139, 144]


This dataset contains the token sequences, but some of these are longer than the maximum input length for the model.

You can now use a second preprocessing function to

* concatenate all the sequences
* split the concatenated sequences into shorter chunks defined by block_size, which should be both shorter than the maximum input length and short enough for your GPU RAM.

In [31]:
block_size = 128

def group_texts(examples):
  # Concatenate all texts.
  concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
  total_length = len(concatenated_examples[list(examples.keys())[0]])
  # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
  # customize this part to your needs.
  if total_length >= block_size:
    total_length = (total_length // block_size) * block_size
  # Split by chunks of block_size.
  result = {
    k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
    for k, t in concatenated_examples.items()
  }
  result["labels"] = result["input_ids"].copy()
  return result

In [32]:
lm_dataset = tokenized_eli5.map(group_texts, batched=True, num_proc=4)

Map (num_proc=4):   0%|          | 0/4000 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/1000 [00:00<?, ? examples/s]

Now create a batch of examples using DataCollatorForLanguageModeling. It’s more efficient to dynamically pad the sentences to the longest length in a batch during collation, instead of padding the whole dataset to the maximum length.

In [33]:
from transformers import DataCollatorForLanguageModeling

tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

## Train

In [34]:
from transformers import AutoModelForCausalLM, TrainingArguments, Trainer

model = AutoModelForCausalLM.from_pretrained("distilbert/distilgpt2")

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

At this point, only three steps remain:

1. Define your training hyperparameters in TrainingArguments. The only required parameter is output_dir which specifies where to save your model. You’ll push this model to the Hub by setting push_to_hub=True (you need to be signed in to Hugging Face to upload your model).
1. Pass the training arguments to Trainer along with the model, datasets, and data collator.
1. Call train() to finetune your model.

In [42]:
training_args = TrainingArguments(
  output_dir="my_awesome_eli5_clm-model",
  eval_strategy="epoch",
  learning_rate=2e-5,
  weight_decay=0.01,
  push_to_hub=False,
  report_to="none",
)

In [43]:
trainer = Trainer(
  model=model,
  args=training_args,
  train_dataset=lm_dataset["train"],
  eval_dataset=lm_dataset["test"],
  data_collator=data_collator,
  tokenizer=tokenizer,
)

In [44]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,3.942900,3.828583
2,3.855000,3.817937
3,3.822800,3.816746


TrainOutput(global_step=3942, training_loss=3.8788144959827657, metrics={'train_runtime': 219.2415, 'train_samples_per_second': 143.828, 'train_steps_per_second': 17.98, 'total_flos': 1029933804552192.0, 'train_loss': 3.8788144959827657, 'epoch': 3.0})

In [45]:
import math

eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 45.46


In [50]:
trainer.save_model('my_model')

## Inference

In [46]:
prompt = "Somatic hypermutation allows the immune system to"

In [52]:
from transformers import pipeline

generator = pipeline("text-generation", model="my_model", device=0)
generator(prompt)

[{'generated_text': "Somatic hypermutation allows the immune system to produce more cells to attack. It makes your body more immune, but doesn't necessarily stop when some part of your body is damaged. The immune system can take a certain amount of damage to try and"}]